# Проект по A/B - тестам.

### `chat_id ` из Телеграма:

In [ ]:
326525586

326525586

------------------

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##  <ins>Задание 1.</ins>

В этом задании вам предстоит провести A/B тест на процессе продажных обзвонов. 

**Контекст**

Мы много платим оператору из-за того, что он сам по каждому заданию звонит клиенту и ждёт ответа клиента. Мы хотим попробовать снизить затраты на операторов, внедрив систему, которая будет сама звонить человеку и в случае успеха перенаправлять задание оператору.
        
**Тест**

Мы проводим тест, в рамках которого мы разбиваем весь поток на две части рановероятно, где одной части (контролю) мы продаём  без автоматической системы дозвона, а второй части (тесту) - с ней. Целью этого теста является снижение расходов на задание.

**Статистические вводные**

Уровень значимости 5%.
Мы хотели бы различать 5%-ное изменение целевой метрики с вероятностью 90%.

**Описание данных**
Процесс продажи устроен следующим образом:
1. создаётся задание на звонок человеку;
2. оператор берёт задание и пытается дозвониться до человека;
3. если не дозванивается, мы несём некоторые затраты на время оператора и поддержку инфраструктуры;
4. если дозванивается, пытается продать наш продукт;
5. если продал, мы имеем некоторый заработок (PV) и повышенные расходы (в том числе на вознаграждение оператора).

Описание колонок в исторических данных.
* ID - номер задания на звонок
* Флаг дозвона - 1 если в рамках задания мы дозвонились человеку, иначе 0
* Флаг продажи - 1 если в рамках задания мы продали наш продукт человеку, иначе 0
* Расходы - сумма затрат на задание
* PV - 0 если мы не продали наш продукт, иначе = PV проданного продукта
* NPV = PV - Расходы

Алгоритм решения задачи следующий.
1. Определите целевую метрику (= какой-то из колонок) и привести аргументацию
Определить альтернативу.
2. Исходя из исторических данных, определите минимальный размер n одной выборки. Привести аргументацию и написать как получилось то или иное число.
3. Примените критерий и сделайте вывод об успешности или неуспешности теста. Расписать подробно с аргументами.


In [ ]:
hist_telesales = pd.read_csv('/content/drive/MyDrive/tinkoff-project/hist_telesales.csv')
hist_telesales

,ID,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
0,0,1,0,90,0,-90
1,1,0,0,5,0,-5
2,2,0,0,68,0,-68
3,3,1,0,22,0,-22
4,4,1,0,22,0,-22
...,...,...,...,...,...,...
72156,72156,1,1,577,1346,769
72157,72157,0,0,8,0,-8
72158,72158,0,0,23,0,-23
72159,72159,0,0,4,0,-4


### 0. Запосить у бота в Телеграме условие командой `/get_project1_desc`.

### 1. Необходимо выбрать метрику и привести аргументацию.

Целевой метрикой для данного A/B теста является **NPV (чистая прибыль)**, так как наша задача - снижение расходов на задание без ущерба для продаж. Мы хотим определить, снижает ли автоматическая система дозвона расходы и увеличивает NPV, не влияя на продажи.


### 2. Альтернатива в критерии.

- **Нулевая гипотеза (H0)**: автоматическая система дозвона не влияет на NPV.
- **Альтернативная гипотеза (H1)**: автоматическая система дозвона увеличивает NPV на 5% или более.

### 3.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

Загрузим исторические данные. Определим среднее значение (µ) и стандартное отклонение (σ) для NPV.

Определим уровень значимости (α) и мощность критерия (1 - β). В данном случае, α = 0.05 и мощность критерия равна 90%, что означает β = 1 - 0.9 = 0.1.

Рассчитаем допустимую ошибку (Δ) для изменения в 5% от среднего значения NPV:
- Δ  = 0.05 * µ

Найдем Z-оценки для заданных уровней значимости (Zα/2) и мощности критерия (Zβ):
- Zα/2 ≈ 1.96 (при α = 0.05)
- Zβ ≈ 1.28 (при β = 0.1)

Используем формулу для расчета минимального размера выборки (n):
- n = ((Zα/2 + Zβ) * σ / Δ)²

где Zα/2 - квантиль нормального распределения для уровня значимости α/2, Zβ - квантиль нормального распределения для уровня мощности 1-β, σ - стандартное отклонение NPV, Δ - допустимая ошибка.

Округлим полученный результат вверх до ближайшего целого числа, чтобы получить минимальный размер выборки (n) для каждой группы (контрольной и тестовой).

In [ ]:
def calculate_sample_size(data, alpha=0.05, beta=0.1, target_change=0.05):
    # Расчет среднего значения и стандартного отклонения для NPV
    mean_npv = data['NPV'].mean()
    std_npv = data['NPV'].std()
    
    # Расчет допустимой ошибки (E)
    delta = target_change * mean_npv
    
    # Расчет Z-оценок для заданных уровней значимости и мощности
    z_alpha_2 = stats.norm.ppf(1 - alpha / 2)
    z_beta = stats.norm.ppf(1 - beta)
    
    # Расчет минимального размера выборки (n)
    n = math.ceil(((z_alpha_2 + z_beta) * std_npv / delta) ** 2)
    
    return n


In [ ]:
sample_size = calculate_sample_size(hist_telesales)
print(f"Минимальный размер выборки: {sample_size}")

Минимальный размер выборки: 24365


### 3.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project1_sample {размер выборки}` и вписать <ins>код выборки</ins>:

Код выборки: `10066`

In [ ]:
control = pd.read_csv('/content/drive/MyDrive/tinkoff-project/Контроль.csv') 
test = pd.read_csv('/content/drive/MyDrive/tinkoff-project/Тест.csv') 


In [ ]:
control

,ID,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
0,0,0,0,14,0,-14
1,1,1,0,70,0,-70
2,2,1,1,1696,2409,713
3,3,1,0,148,0,-148
4,4,0,0,26,0,-26
...,...,...,...,...,...,...
24360,24360,1,1,511,1400,889
24361,24361,0,0,166,0,-166
24362,24362,0,0,52,0,-52
24363,24363,0,0,102,0,-102


In [ ]:
test

,ID,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
0,0,1,0,16,0,-16
1,1,1,0,1,0,-1
2,2,0,0,33,0,-33
3,3,1,0,13,0,-13
4,4,1,0,66,0,-66
...,...,...,...,...,...,...
24360,24360,1,0,149,0,-149
24361,24361,0,0,60,0,-60
24362,24362,0,0,25,0,-25
24363,24363,0,0,1,0,-1


### 4. Принятие решения. Расписать подробно с аргументами.

После проведения эксперимента сравним среднее значение NPV для контрольной и тестовой группы. Для проверки статистической значимости различий между группами применим t-критерий Стьюдента.

Если p-value, полученное в результате t-теста, меньше уровня значимости (0.05), то отвергаем нулевую гипотезу (H0) в пользу альтернативной (H1). В этом случае можно сделать вывод о том, что автоматическая система дозвона успешно снижает расходы и увеличивает NPV на 5% или более.

In [ ]:

def run_ab_test(kontrol, test, alpha=0.05, positive_conclusion="", 
                negative_conlusion="", success_test=""):
    # Выполнение t-теста на двух независимых выборках
    t_stat, p_value = stats.ttest_ind(kontrol['NPV'], test['NPV'],
                                      equal_var=False)
    
    # Вывод результатов t-теста
    print(f"t-статистика: {t_stat}")
    print(f"p-значение: {p_value}")
    
    # Принятие решения на основе p-значения и уровня значимости
    if p_value < alpha:
        print(f"Отвергаем нулевую гипотезу: {positive_conclusion}.")
        print(f"Тест успешен: {success_test}.")
    else:
        print(f"Не удалось отвергнуть нулевую гипотезу:{negative_conlusion}.")
        print("Тест неуспешен.")


In [ ]:
run_ab_test(control, test,
            positive_conclusion="автоматическая система дозвона влияет на NPV",
            negative_conlusion="автоматическая система дозвона \
не оказывает значимого влияния на NPV",
            success_test="автоматическая система дозвона \
увеличивает NPV на 5% или более")

t-статистика: -2.4961796798489395
p-значение: 0.012557182168011832
Отвергаем нулевую гипотезу: автоматическая система дозвона влияет на NPV.
Тест успешен: автоматическая система дозвона увеличивает NPV на 5% или более.


Тест успешен: автоматическая система дозвона увеличивает NPV на 5% или более.


Мы видим, что p-значение меньше уровня значимости (α = 0.05). Это означает, что мы можем отвергнуть нулевую гипотезу о том, что автоматическая система дозвона не влияет на NPV.

Таким образом, вывод следующий: тест успешен, и автоматическая система дозвона увеличивает NPV на 5% или более. Это говорит о том, что внедрение автоматической системы дозвона может помочь снизить расходы на операторов и повысить эффективность продажных обзвонов.

------------------

##  <ins>Задание 2.</ins>

В этом задании вам предстоит провести A/B тест на процессе оформления кредитных карт. 
**Контекст**

ЦБ смягчило правило, ограничивающее варианты продажи услуги. Мы хотим попробовать более агрессивную продажу услуги, но боимся, что это повлияет на экономику основного продукта (КК) из-за негатива клиентов.

**Тест**

Мы проводим тест, в рамках которого мы разбиваем весь поток на две части рановероятно, где одной части (контролю)  мы будем предлагать услугу текущим способом, а второй части (тесту) - новым более агрессивным способом. Целью этого теста является увеличение доходности КК + услуги.

**Статистические вводные**

Проверка однородности выбранного параметра осуществяется с уровнем значимости 2%.

Уровень значимости 5%.
Мы хотели бы различать 3%-ное изменение целевой метрики с вероятностью 85%.


**Описание данных**

Процесс оформления кредитной карты (КК) устроен следующим образом:

1. человек заходит на сайт и заполняет информацию о себе (в том числе возраст и доход клиента) для получения кредитной карты;
2. человек проходит скоринг (в том числе в рамках которого оценивается вероятность банкротства клиента);
3. создаётся заявка на оформление кредитной карты;
4. человеку выпускается виртуальная карта, а также предлагается наша услуга к этому счёту;
5. с некоторой вероятностью человек утилизирует счёт (начинает им пользоваться);
6. если человек не утилизирует счёт, мы несём некоторые затраты на время оператора и поддержку инфраструктуры;
7. если человек утилизировал карту, мы имеем некоторый заработок (PV) и повышенные расходы.

Описание колонок в исторических данных.

* ID - номер заявки, прошедшей скоринг
* Возраст - возраст клиента, указанный при оформлении КК
* Доход клиента - доход клиента, указанный при оформлении КК
* Вероятность банкротства - оценка вероятности банкротства клиента, полученная с помощью скоринга по параметрам клиента
* Флаг утилизация счёта - совершение первой расходной операции на счёте КК
* Расходы - сумма затрат на заявку
* PV КК - 0 если счёт не утилизирован, иначе = PV кредитной карты
* PV услуги - 0 если счёт не утилизирован, иначе = PV этой услуги
* NPV = PV КК + PV услуги - Расходы


Алгоритм решения задачи следующий.
1. Определите целевую метрику (= какой-то из колонок) и привести аргументацию.
2. Определить альтернативу в критерии.
3. Нужно выбрать параметр(-ы), влияющий(-ие) на целевую метрику. Привести аргументацию.
4 Исходя из исторических данных, определите минимальный размер n одной выборки. Привести аргументацию и написать как получилось то или иное число.
5. Проверьте однородность выборок по выбранному в пункте 4 параметру.  Расписать подробно с аргументами.
6. Если выборки неоднородны, то тест провален. Если выборки однородны, примените критерий для целевой метрики.  Расписать подробно с аргументами.
7. Опишите ваши выводы по тесту.  Принятие решения. Расписать подробно с аргументами.


In [ ]:
hist_credit_card = pd.read_csv('/content/drive/MyDrive/tinkoff-project/hist_credit_card.csv')
hist_credit_card

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,19,21620.835463,0.138061,0,102,0,0,-102
1,1,27,24897.990863,0.035508,1,409,11686,1754,13031
2,2,50,23989.526947,0.098793,0,16,0,0,-16
3,3,18,38442.409756,0.365661,1,788,13738,1578,14528
4,4,24,21291.521612,0.036909,1,1048,6594,2213,7759
...,...,...,...,...,...,...,...,...,...
123250,123250,32,26099.633927,0.110756,0,47,0,0,-47
123251,123251,20,24579.749275,0.113920,1,594,14268,1672,15346
123252,123252,36,34062.902531,0.247122,1,77,5950,2017,7890
123253,123253,67,24609.838522,0.020752,1,279,6278,1847,7846


### 0. Запосить у бота в Телеграме условие командой `/get_project2_desc`.

### 1. Необходимо выбрать метрику и привести аргументацию.

**Целевая метрика: NPV** (чистая приведенная стоимость). NPV учитывает доходность от кредитной карты и услуги, а также расходы на заявку. Это позволяет оценить влияние нового способа продажи услуги на общую экономическую эффективность.


### 2. Альтернатива в критерии.

- **Альтернативная гипотеза (H1)**: более агрессивный способ продажи услуги приводит к изменению NPV на 3% или более. 
- **Нулевая гипотеза (H0)**: изменение NPV составляет менее 3%.

### 3. Нужно выбрать параметр(-ы), влияющий(-ие) на целевую метрику. Привести аргументацию.

Вероятно, на NPV влияют следующие параметры:
 - Возраст: Возраст клиента **может влиять** на его платежеспособность, стабильность работы и финансовую дисциплину. Молодые клиенты могут быть менее стабильными, что может привести к неплатежам или меньшему использованию кредитной карты, что влияет на NPV. Пожилые клиенты могут быть более консервативными в отношении использования кредитных карт, что также может повлиять на NPV.

- Доход клиента: Доход клиента **напрямую влияет** на его платежеспособность и возможность возвращать кредиты. Клиенты с более высоким доходом могут с большей вероятностью использовать кредитные карты и успешно обслуживать кредиты, что положительно скажется на NPV. Клиенты с низким доходом могут испытывать затруднения с возвращением кредитов, что негативно скажется на NPV.

- Вероятность банкротства: Вероятность банкротства клиента **является одним из ключевых факторов риска** для банка. Клиенты с высокой вероятностью банкротства могут не справиться с кредитными обязательствами, что приведет к потере NPV для банка. Клиенты с низкой вероятностью банкротства, наоборот, с большей вероятностью будут обслуживать кредиты и использовать кредитные карты, что увеличит NPV.

Эти параметры влияют на вероятность утилизации счета, а также на возможные доходы и расходы, связанные с кредитными картами и услугами, и, следовательно, влияют на общий NPV. Возьмём **вероятность банкротства** как самую главную.


### 4.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

Определим среднее значение (µ) и стандартное отклонение (σ) для NPV.

Определим уровень значимости (α) и мощность критерия (1 - β). В данном случае, α = 0.05 и мощность критерия равна 90%, что означает β = 1 - 0.85 = 0.15.

Рассчитаем допустимую ошибку (Δ) для изменения в 3% от среднего значения NPV:
- Δ  = 0.03 * µ

Найдем Z-оценки для заданных уровней значимости (Zα/2) и мощности критерия (Zβ):
- Zα/2 ≈ 1.96 (при α = 0.05)
- Zβ ≈ 1.04 (при β = 0.15)

Используем формулу для расчета минимального размера выборки (n):
- n = ((Zα/2 + Zβ) * σ / Δ)²

где Zα/2 - квантиль нормального распределения для уровня значимости α/2, Zβ - квантиль нормального распределения для уровня мощности 1-β, σ - стандартное отклонение NPV, Δ - допустимая ошибка.

Округлим полученный результат вверх до ближайшего целого числа, чтобы получить минимальный размер выборки (n) для каждой группы (контрольной и тестовой).

In [ ]:
sample_size = calculate_sample_size(hist_telesales, alpha=0.05, beta=0.15, target_change=0.03)
print(f"Минимальный размер выборки: {sample_size}")

Минимальный размер выборки: 57831


### 4.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project2_sample {размер выборки}` и вписать <ins>код выборки</ins>:

Код выборки: `63257`.

In [ ]:
control = pd.read_csv('/content/drive/MyDrive/tinkoff-project/задание2/Контроль.csv') 
test = pd.read_csv('/content/drive/MyDrive/tinkoff-project/задание2/Тест.csv') 


In [ ]:
control

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,43,26576.011499,0.052171,1,241,8409,3034,11202
1,1,75,26361.901975,0.025839,1,643,7122,2663,9142
2,2,61,60260.264467,0.082389,1,334,5246,1388,6300
3,3,42,23192.833027,0.037824,1,251,6192,1856,7797
4,4,57,21438.514437,0.102655,0,2,0,0,-2
...,...,...,...,...,...,...,...,...,...
57826,57826,29,23068.297986,0.413482,1,1565,5907,1576,5918
57827,57827,59,29418.031631,0.105694,1,85,8223,1426,9564
57828,57828,64,21426.440637,0.052161,1,2152,5820,1577,5245
57829,57829,44,31115.351909,0.077696,1,250,6178,1422,7350


In [ ]:
test

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,70,20725.570610,0.101303,1,1519,4940,3776,7197
1,1,44,26807.928357,0.154720,1,253,6562,2387,8696
2,2,38,28478.851101,0.098137,0,0,0,0,0
3,3,50,29883.631236,0.164273,1,116,7299,2620,9803
4,4,52,31297.374022,0.024047,1,162,8462,3412,11712
...,...,...,...,...,...,...,...,...,...
57826,57826,60,23148.585607,0.269336,0,36,0,0,-36
57827,57827,48,47423.657545,0.304969,0,60,0,0,-60
57828,57828,45,32248.284344,0.055145,1,677,5188,5898,10409
57829,57829,38,20133.479547,0.043727,1,751,7603,2397,9249


### 5. Проверка на однородность, применение критерия. Принятие решения. Расписать подробно с аргументами.

#### 5.1. Проверка на однородность

In [ ]:

def check_homogeneity(control, test, alpha=0.05):
    control_prob = control['Вероятность банкротства']
    test_prob = test['Вероятность банкротства']
    
    ttest_result = stats.ttest_ind(control_prob, test_prob, equal_var=True)
    
    print("Двухвыборочный t-критерий Стьюдента для \
сравнения вероятности банкротства контрольной и тестовой выборок:", ttest_result)
    
    if ttest_result.pvalue < alpha:
        print(f"Так как p-значение {ttest_result.pvalue} меньше \
уровня значимости {alpha}, мы отвергаем нулевую гипотезу о том, \
что выборки однородны и вероятность банкротства не изменилась.")
    else:
        print(f"Так как p-значение {ttest_result.pvalue} больше уровня \
значимости {alpha}, мы не можем отвергнуть нулевую гипотезу о том,\
что выборки однородны и вероятность банкротства не изменилась.")



In [ ]:
check_homogeneity(control, test)


Двухвыборочный t-критерий Стьюдента для сравнения вероятности банкротства контрольной и тестовой выборок: Ttest_indResult(statistic=0.20707348053239402, pvalue=0.8359528372283845)
Так как p-значение 0.8359528372283845 больше уровня значимости 0.05, мы не можем отвергнуть нулевую гипотезу о том,что выборки однородны и вероятность банкротства не изменилась.


#### 5.2 Применения критерия для целевой метрики.

Теперь, когда мы убедились, что выборки однородны, мы можем сравнить их для оценки влияния изменения способа продажи услуги на NPV.


1. Нулевая гипотеза (H0) состоит в том, что средние значения NPV контрольной и тестовой выборок равны, то есть изменение способа продажи услуги не влияет на NPV.
2. Альтернативная гипотеза (H1) состоит в том, что средние значения NPV контрольной и тестовой выборок отличаются, то есть изменение способа продажи услуги влияет на NPV.

In [ ]:
run_ab_test(control, test,
            positive_conclusion="изменение способа продажи услуги \
влияет на NPV",
            negative_conlusion="изменение способа продажи услуги \
не влияет на NPV",
            success_test="более агрессивный способ продажи услуги приводит \
к изменению NPV на 3% или более")

t-статистика: -4.215940212960836
p-значение: 2.4892976313175556e-05
Отвергаем нулевую гипотезу: изменение способа продажи услуги влияет на NPV.
Тест успешен: более агрессивный способ продажи услуги приводит к изменению NPV на 3% или более.
